In [12]:
import requests
import json
import random
import time
import pickle

model1 = "dRIT_pi_SNLI_fs1"
model5 = "dRIT_pi_SNLI_fs5"
model10 = "dRIT_pi_SNLI_fs10"
model25 = "dRIT_pi_SNLI_fs25"
model50 = "dRIT_pi_SNLI_fs50"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TEST de SNLI

In [13]:
import pandas as pd

In [14]:
df_t=pd.read_pickle("../predictions/RL_SNLI_TEST_600.pickle")

In [15]:
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,1,0.000000,0.0,1.000000,0.666667,0.182158,4.758932,1.000000,1.000000,0,0,0.773148,0,0,1.000000,entailment,A small child with a backpack walks down a nar...,A small child is walking on a road
1,2,0.333333,0.0,1.000000,0.589256,0.140694,4.613125,1.000000,1.000000,0,0,0.464286,0,0,0.666667,entailment,A brown and white dog walking away from a blac...,A dog stands in the water.
2,2,0.666667,0.0,0.333333,0.288675,0.108599,4.038265,0.681953,0.666667,1,0,0.555556,0,0,0.333333,entailment,A man in the bucket of a chery-picker cuts dow...,A man is trimming trees.
3,2,0.666667,0.0,0.500000,0.377964,-0.049452,4.145515,0.535615,0.666667,1,0,0.464286,0,0,0.333333,entailment,Three people are making transactions at an aut...,A line of people at an ATM.
4,2,0.666667,0.0,0.000000,0.154303,0.108264,3.879840,0.510767,1.000000,2,0,0.436508,0,0,0.333333,entailment,A woman looks at a picture and framing tools w...,There is a person in a room.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.750000,0.894427,0.231952,4.379169,0.776339,0.600000,0,0,0.933333,0,0,0.600000,contradiction,There are two people walking in the sand.,There are five people walking in the sand with...
596,2,0.333333,0.0,0.400000,0.524142,0.164122,4.365030,0.806454,1.000000,2,0,0.543040,0,0,0.666667,contradiction,A man with a gray beard and a little boy are s...,The man and the boy are taking a nap on the bu...
597,2,0.750000,0.0,0.333333,0.404520,0.105212,4.266216,0.610287,0.750000,1,0,0.263636,0,0,0.250000,contradiction,"A race car, numbered 104 is rounding a bend as...",People are watching a ball game.
598,2,1.000000,0.0,0.000000,0.154303,-0.003144,3.861699,0.143365,0.250000,1,0,0.311111,0,0,0.000000,contradiction,A woman in a white dress with a white umbrella...,Two sisters are flipping each other off.


In [16]:
df_t.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

# RIT pi fs 1

In [17]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model1,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_pifs1_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# RIT pi fs 5

In [18]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model5,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("../ollama3.1_dRIT_pi_fs5/resultados/rit_pifs5_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# RIT pi fs 10

In [19]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model10,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("../ollama3.1_dRIT_pi_fs10/resultados/rit_pifs10_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# RIT pi fs 25

In [20]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model25,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("../ollama3.1_dRIT_pi_fs25/resultados/rit_pifs25_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# RIT pi fs 50

In [21]:
lista_respuestasOllama=[]
faltantes=[]
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        #print(strings["sentence1"],strings["sentence2"],strings['gold_label'])
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        plus next features:
        sums:'''+ str(strings["sumas"])+'''
        negH:'''+ str(strings["negH"])+'''
        list_compability:'''+ str(strings["list_comp"])+'''
        list_incompability:'''+ str(strings["list_incomp"])+'''
        relation:'''+ str(strings["relation"])+'''
        entail:'''+ str(strings["entail"])+'''
        negT:'''+ str(strings["negT"])+'''
        mutinf:'''+ str(strings["mutinf_t"])+'''
        contradiction:'''+ str(strings["contradiction"])+'''
        no_matcheadas:'''+ str(strings["no_matcheadas"])+'''
        Jaro-Winkler_rit:'''+ str(strings["Jaro-Winkler_rit"])+'''
        simBoW:'''+ str(strings["simBoW"])+'''
        jaccard:'''+ str(strings["jaccard"])+'''
        overlap_ent:'''+ str(strings["overlap_ent"])+'''
        max_info_t:'''+ str(strings["max_info_t"])+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model50,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("../ollama3.1_dRIT_pi_fs50/resultados/rit_pifs50_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27